# Batch Transform Inference

## Deploy Model from Saved Assets

In [ ]:
import boto3
import time, random, uuid
import pandas as pd
import numpy as np
from sagemaker import Model
from sagemaker import image_uris

s3_bucket = "INSERT_S3_BUCKET"
model_artifact_s3_key = "INSERT_S3_KEY"
sagemaker_role = "INSERT_ARN"
model_artifact_s3 = f"s3://{s3_bucket}/{model_artifact_s3_key}"

# Initialize Boto3 SageMaker client
sagemaker_client = boto3.client("sagemaker", region_name="us-east-1")
xgboost_image_uri = image_uris.retrieve(framework='xgboost',region='us-east-1', version='1.7-1')


# **Create a SageMaker Model**
model_name = f"xgboost-model-{int(time.time())}"
print("Creating model in SageMaker named: {}".format(model_name))
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        "Image": xgboost_image_uri,
        "ModelDataUrl": model_artifact_s3,
    },
    ExecutionRoleArn=sagemaker_role,
)


# **Deploy the Model as an Endpoint**
# Create the endpoint configuration
endpoint_config_name = f"xgboost-endpoint-config-{int(time.time())}"
endpoint_name = f"xgboost-endpoint-{int(time.time())}"

# Create Endpoint Configuration
print("Creating endpoint configuration...")
sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "XGBoostVariant1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
        }
    ],
)

# **Deploy the Model as an Endpoint**
# Create Endpoint
print("Deploying model as an endpoint...")
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

# Wait for endpoint to be ready
print("Waiting for endpoint to be ready...")
while True:
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = response["EndpointStatus"]
    if status in ["InService", "Failed"]:
        print(f"Endpoint Status: {status}")
        break
    time.sleep(10)

# Check if deployment was successful
if status != "InService":
    raise Exception(f"Deployment failed with status: {status}")

print(f"Model deployed successfully at endpoint: {endpoint_name}")


## Batch Transform
Once the model is deployed, we can package up and send a batch transform job.

In [ ]:
# Generate large dataset for prediction query and store in S3

s3_batch_storage_prefix = "batch-transform"  # Folder inside the bucket

# Generate Random CSV Data
num_rows = 10000  # Adjust as needed
df = pd.DataFrame({
    "age": np.random.randint(18, 81, size=num_rows),
    "edu": np.random.randint(0, 5, size=num_rows),
    "income": np.random.randint(20000, 150001, size=num_rows)
})

# Save CSV Locally
uniqueID = uuid.uuid4().hex
csv_filename = f"random_data_{uniqueID}.csv"
df.to_csv(csv_filename, index=False, header=False)

# Upload CSV to S3
s3_client = boto3.client("s3")
s3_batch_transform_key = f"{s3_batch_storage_prefix}/{csv_filename}"
s3_client.upload_file(csv_filename, s3_bucket, s3_batch_transform_key)

print(f"Uploaded file to s3://{s3_bucket}/{s3_batch_transform_key}")

In [ ]:
# Create SageMaker Batch Transform Job

job_name = f"batch-transform-{uuid.uuid4().hex}"
response = sagemaker_client.create_transform_job(
    TransformJobName=job_name,
    ModelName=model_name,
    BatchStrategy="SingleRecord",  # or 'MultiRecord' for multiple rows per batch
    TransformInput={
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": f"s3://{s3_bucket}/{s3_batch_transform_key}"
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line"
    },
    TransformOutput={
        "S3OutputPath": f"s3://{s3_bucket}/{s3_batch_storage_prefix}/output_{uniqueID}"
    },
    TransformResources={
        "InstanceType": "ml.m5.large",  # Change as needed
        "InstanceCount": 1
    }
)

print(f"Started batch transform job: {job_name}")